In [1]:
import pandas as pd
from konlpy.tag import Okt
from collections import Counter
import re

# 데이터 로드
data = pd.read_csv("merged_data_with_categories.csv")

# Okt 형태소 분석기 초기화
okt = Okt()

# 리뷰 데이터 전처리: "리뷰 없음" 제거
data = data[data["review_content"] != "리뷰 없음"]

# 리뷰 데이터에서 중복 제거
data = data.drop_duplicates(subset=["review_content"])

# 불용어 리스트 정의
stopwords = [
    "의", "가", "이", "은", "들", "는", "좀", "잘", "걍", "과", "도", "를", "으로", "자", "에", "와",
    "한", "하다", "것", "라고", "에게", "라면", "을", "이라", "라니", "있다", "아", "랑", "쯤된", "에서",
    "에선", "어", "이지만", "으로나", "때", "때는", "때라면", "때라서", "라", "이다", "있", "죠", "고",
    "니", "로", "같", "어서", "어요", "는데", "습니다", "면서", "많이", "마", "더", "그렇다", "당", "안",
    "정말", "같다", "임", "만", "인", "부터", "저", "우리", "너", "저희", "그", "수", "아무", "나",
    "너희", "제", "책", "리뷰", "배송", "소설", "좋다", "작품", "뿐", "거", "중", "입니다", "추합니다",
]

# 리뷰 데이터 전처리 함수
def preprocess_review(review):
    review = re.sub(r"[^가-힣\s]", "", str(review))  # 한글과 공백만 남기기
    return review

# 명사-형용사 조합 추출 함수
def extract_noun_adj_pairs(text):
    tokens = okt.pos(text)  # 형태소 및 품사 태깅
    filtered_tokens = [
        (word, tag) for word, tag in tokens if word not in stopwords
    ]  # 불용어 제거
    pairs = []
    for i in range(len(filtered_tokens) - 1):
        if filtered_tokens[i][1] == "Noun" and filtered_tokens[i + 1][1] == "Adjective":
            pairs.append((filtered_tokens[i][0], filtered_tokens[i + 1][0]))
    return pairs

# 데이터 전처리 및 명사-형용사 조합 추출
data["cleaned_review"] = data["review_content"].apply(preprocess_review)
data["noun_adj_pairs"] = data["cleaned_review"].apply(extract_noun_adj_pairs)

# 모든 리뷰에서 추출된 명사-형용사 조합 리스트 생성
all_pairs = [pair for sublist in data["noun_adj_pairs"] for pair in sublist]

# 명사-형용사 조합 빈도 계산
pair_counts = Counter(all_pairs)

# 자주 사용된 조합 추출 (상위 20개)
common_pairs = pair_counts.most_common(20)

# 조합별 리뷰와 리뷰 개수 추출
pair_reviews = []
for pair, _ in common_pairs:
    pair_pattern = f"{pair[0]}.*{pair[1]}"  # 명사와 형용사가 연속적으로 등장하는 패턴
    matching_reviews = data[data["cleaned_review"].str.contains(pair_pattern, na=False)]
    pair_reviews.append((pair, len(matching_reviews)))

# 리뷰 개수로 내림차순 정렬
sorted_pair_reviews = sorted(pair_reviews, key=lambda x: x[1], reverse=True)

# 출력
print("Top Noun-Adjective Pairs Sorted by Review Count:")
for pair, review_count in sorted_pair_reviews:
    print(f"Pair: {pair}, Review Count: {review_count}")


Top Noun-Adjective Pairs Sorted by Review Count:
Pair: ('알', '있는'), Review Count: 795
Pair: ('볼', '있는'), Review Count: 599
Pair: ('살', '있는'), Review Count: 563
Pair: ('게', '아니라'), Review Count: 518
Pair: ('볼', '있어'), Review Count: 492
Pair: ('알', '있어서'), Review Count: 304
Pair: ('볼', '있어서'), Review Count: 273
Pair: ('관심', '있는'), Review Count: 265
Pair: ('보고', '있습니다'), Review Count: 252
Pair: ('가지', '있는'), Review Count: 243
Pair: ('가치', '있는'), Review Count: 224
Pair: ('보고', '있어요'), Review Count: 211
Pair: ('의미', '있는'), Review Count: 199
Pair: ('깊이', '있는'), Review Count: 193
Pair: ('볼', '있게'), Review Count: 187
Pair: ('꼭', '필요한'), Review Count: 160
Pair: ('가장', '좋은'), Review Count: 113
Pair: ('아주', '좋은'), Review Count: 107
Pair: ('수가', '없다'), Review Count: 95
Pair: ('깊이', '있게'), Review Count: 84


In [2]:
# 형용사-명사 조합 추출 함수
def extract_adj_noun_pairs(text):
    tokens = okt.pos(text)  # 형태소 및 품사 태깅
    filtered_tokens = [
        (word, tag) for word, tag in tokens if word not in stopwords
    ]  # 불용어 제거
    pairs = []
    for i in range(len(filtered_tokens) - 1):
        if filtered_tokens[i][1] == "Adjective" and filtered_tokens[i + 1][1] == "Noun":
            pairs.append((filtered_tokens[i][0], filtered_tokens[i + 1][0]))
    return pairs

# 데이터 전처리 및 형용사-명사 조합 추출
data["cleaned_review"] = data["review_content"].apply(preprocess_review)
data["adj_noun_pairs"] = data["cleaned_review"].apply(extract_adj_noun_pairs)

# 모든 리뷰에서 추출된 형용사-명사 조합 리스트 생성
all_adj_noun_pairs = [pair for sublist in data["adj_noun_pairs"] for pair in sublist]

# 형용사-명사 조합 빈도 계산
pair_counts = Counter(all_adj_noun_pairs)

# 자주 사용된 조합 추출 (상위 20개)
common_pairs = pair_counts.most_common(20)

# 출력
print("Top Adjective-Noun Pairs Sorted by Review Count:")
for pair, count in common_pairs:
    print(f"Pair: {pair}, Count: {count}")

Top Adjective-Noun Pairs Sorted by Review Count:
Pair: ('많은', '도움'), Count: 389
Pair: ('같은', '느낌'), Count: 227
Pair: ('있는', '사람'), Count: 212
Pair: ('많은', '사람'), Count: 196
Pair: ('많은', '생각'), Count: 193
Pair: ('좋은', '글'), Count: 154
Pair: ('좋아하는', '사람'), Count: 134
Pair: ('좋은', '내용'), Count: 133
Pair: ('아름다', '움'), Count: 131
Pair: ('빨간', '머리'), Count: 128
Pair: ('있는', '내용'), Count: 115
Pair: ('많은', '분'), Count: 107
Pair: ('좋아하는', '작가'), Count: 106
Pair: ('있다고', '생각'), Count: 103
Pair: ('있는', '시간'), Count: 94
Pair: ('미야', '베'), Count: 94
Pair: ('좋을', '듯'), Count: 91
Pair: ('있는', '기회'), Count: 91
Pair: ('자세한', '설명'), Count: 83
Pair: ('아쉬운', '점'), Count: 81


In [3]:
# 명사-명사 조합 추출 함수
def extract_noun_noun_pairs(text):
    tokens = okt.pos(text)  # 형태소 및 품사 태깅
    filtered_tokens = [
        (word, tag) for word, tag in tokens if word not in stopwords
    ]  # 불용어 제거
    pairs = []
    for i in range(len(filtered_tokens) - 1):
        if filtered_tokens[i][1] == "Noun" and filtered_tokens[i + 1][1] == "Noun":
            pairs.append((filtered_tokens[i][0], filtered_tokens[i + 1][0]))
    return pairs

# 데이터 전처리 및 명사-명사 조합 추출
data["cleaned_review"] = data["review_content"].apply(preprocess_review)
data["noun_noun_pairs"] = data["cleaned_review"].apply(extract_noun_noun_pairs)

# 모든 리뷰에서 추출된 명사-명사 조합 리스트 생성
all_noun_noun_pairs = [pair for sublist in data["noun_noun_pairs"] for pair in sublist]

# 명사-명사 조합 빈도 계산
pair_counts = Counter(all_noun_noun_pairs)

# 자주 사용된 조합 추출 (상위 20개)
common_pairs = pair_counts.most_common(20)

# 출력
print("Top Noun-Noun Pairs Sorted by Review Count:")
for pair, count in common_pairs:
    print(f"Pair: {pair}, Count: {count}")

Top Noun-Noun Pairs Sorted by Review Count:
Pair: ('하나', '하나'), Count: 446
Pair: ('바로', '구매'), Count: 427
Pair: ('다시', '한번'), Count: 400
Pair: ('그림', '체'), Count: 318
Pair: ('소장', '가치'), Count: 287
Pair: ('스마트', '스토어'), Count: 269
Pair: ('중간', '중간'), Count: 262
Pair: ('대한', '이야기'), Count: 225
Pair: ('처음', '접'), Count: 199
Pair: ('펀딩', '참여'), Count: 191
Pair: ('또', '다른'), Count: 187
Pair: ('영어', '공부'), Count: 180
Pair: ('다음', '권'), Count: 171
Pair: ('이번', '권'), Count: 166
Pair: ('강력', '추천'), Count: 162
Pair: ('머리', '앤'), Count: 161
Pair: ('북', '펀드'), Count: 160
Pair: ('영화', '보고'), Count: 155
Pair: ('대해', '생각'), Count: 152
Pair: ('마음', '듭니'), Count: 148


In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from konlpy.tag import Okt

# 1. 데이터 로드 및 전처리
data = pd.read_csv("merged_data_without_duplicates.csv")
data = data[data["review_content"] != "리뷰 없음"]  # "리뷰 없음" 제거

# 중복된 리뷰 내용 제거
data = data.drop_duplicates(subset=["review_content"]).reset_index(drop=True)

okt = Okt()

# 2. 형태소 분석을 통한 키워드 추출
def extract_keywords(text):
    tokens = okt.pos(text)
    keywords = [word for word, tag in tokens if tag in ["Noun", "Adjective"]]
    return " ".join(keywords)

data["processed_review"] = data["review_content"].apply(extract_keywords)

# 3. TF-IDF 계산
vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(2, 3))
tfidf_matrix = vectorizer.fit_transform(data["processed_review"])
terms = vectorizer.get_feature_names_out()

# 4. KMeans 클러스터링
kmeans = KMeans(n_clusters=10, random_state=0).fit(tfidf_matrix)
data["cluster"] = kmeans.labels_

# 5. 각 클러스터에서 대표 키워드 추출
def get_top_keywords(cluster, n_terms=5):
    cluster_indices = data[data["cluster"] == cluster].index
    cluster_tfidf = tfidf_matrix[cluster_indices.tolist()].toarray()
    mean_tfidf = cluster_tfidf.mean(axis=0)
    top_terms = [terms[i] for i in mean_tfidf.argsort()[-n_terms:]]
    return top_terms

top_keywords_per_cluster = {i: get_top_keywords(i) for i in range(kmeans.n_clusters)}

# 6. 결과 출력
print("Top Keywords per Cluster:")
for cluster, keywords in top_keywords_per_cluster.items():
    print(f"Cluster {cluster}: {', '.join(keywords)}")


Top Keywords per Cluster:
Cluster 0: 사람 이해, 생각 다른, 사람 추천, 사람 생각, 다른 사람
Cluster 1: 소장 가치, 많은 도움, 다시 한번, 좋은 입니다, 바로 구매
Cluster 2: 소장 가치, 소장 가치 있는, 가치 있는, 가치 있는 입니다, 있는 입니다
Cluster 3: 생각 듭니, 있어서 좋았습니다, 정말 생각, 가독성 좋고, 중간 중간
Cluster 4: 선물 좋은, 선물 주문, 선물 구입, 선물 구매, 친구 선물
Cluster 5: 초등 학년, 추천 추천, 소식 바로, 보고 바로, 바로 주문
Cluster 6: 한번 야할, 누구 한번, 생각 지금, 초등 학년, 우리 아이
Cluster 7: 생각 좋은, 생각 거리, 생각 입니다, 대해 많은, 많은 생각
Cluster 8: 구입 정말, 추천 입니다, 내용 정말, 있어서 정말, 정말 좋아요
Cluster 9: 이야기 하나, 스마트 스토어, 문장 하나 하나, 문장 하나, 하나 하나


In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
import re

# 1. 데이터 로드 및 전처리
data = pd.read_csv("merged_data_without_duplicates.csv")
data = data[data["review_content"] != "리뷰 없음"]  # "리뷰 없음" 제거

# 중복된 리뷰 제거
data = data.drop_duplicates(subset=["review_content"]).reset_index(drop=True)

# 2. Okt 초기화
okt = Okt()

# 3. 확장된 불용어 리스트 정의
stopwords = [
    # 기본 불용어
    "책", "내용", "저", "것", "거", "수", "때", "듯", "너무", "진짜",
    "정말", "그", "이", "그리고", "제", "더", "좀", "이런", "저런",
    "많이", "많은", "사람", "대한", "좋은", "같다", "보고", "많다", 
    "좋다", "하는", "해서", "함", "이다", "입니다", "하는데", "하는거", 
    "라서", "로", "에서", "이라고", "로서", "라며",

    # 추가된 불용어 (결과에서 발견된 단어들)
    "생각", "구매", "정말", "사람", "마음", "소설", "추천", "작품", 
    "다시", "번역", "대한", "느낌", "이해", "그림", "공부"
]

# 4. 명사 추출 함수 정의 (불용어 제거 포함)
def extract_nouns(text):
    # 한글 전처리: 특수 문자 제거
    text = re.sub(r"[^가-힣\s]", "", str(text))
    # 명사 추출
    nouns = okt.nouns(text)
    # 불용어 제거
    filtered_nouns = [word for word in nouns if word not in stopwords and len(word) > 1]
    return " ".join(filtered_nouns)

# 모든 리뷰에서 명사만 추출
data["processed_review"] = data["review_content"].apply(extract_nouns)

# 5. TF-IDF 계산
vectorizer = TfidfVectorizer(max_features=1000)  # 상위 1000개 명사
tfidf_matrix = vectorizer.fit_transform(data["processed_review"])
terms = vectorizer.get_feature_names_out()

# 6. 명사 키워드의 TF-IDF 점수 계산
tfidf_scores = tfidf_matrix.sum(axis=0).A1  # TF-IDF 점수 합산
keywords = sorted(
    zip(terms, tfidf_scores), key=lambda x: x[1], reverse=True
)  # 점수 기준 내림차순 정렬

# 7. 주요 명사 키워드 출력
top_n_keywords = 20  # 상위 20개 키워드
print("Top Nouns by TF-IDF (After Expanded Stopword Removal):")
for keyword, score in keywords[:top_n_keywords]:
    print(f"Keyword: {keyword}, Score: {score:.2f}")

# 8. CSV 파일로 저장 (옵션)
output_file = "top_nouns_tfidf_filtered.csv"
pd.DataFrame(keywords, columns=["Keyword", "Score"]).to_csv(output_file, index=False)
print(f"Top nouns saved to {output_file}")


Top Nouns by TF-IDF (After Expanded Stopword Removal):
Keyword: 작가, Score: 1741.11
Keyword: 이야기, Score: 1684.64
Keyword: 기대, Score: 1541.13
Keyword: 최고, Score: 837.48
Keyword: 우리, Score: 836.57
Keyword: 사랑, Score: 829.84
Keyword: 구입, Score: 816.59
Keyword: 도움, Score: 815.99
Keyword: 역시, Score: 803.53
Keyword: 하나, Score: 787.04
Keyword: 시간, Score: 773.18
Keyword: 처음, Score: 771.41
Keyword: 다른, Score: 755.19
Keyword: 대해, Score: 751.21
Keyword: 한번, Score: 747.92
Keyword: 설명, Score: 730.29
Keyword: 부분, Score: 724.75
Keyword: 이번, Score: 685.18
Keyword: 영화, Score: 679.41
Keyword: 다음, Score: 678.46
Top nouns saved to top_nouns_tfidf_filtered.csv
